# Implementation of IDPpred
IDPpred method uses three classifiers for predicting intrinsically disordered proteins (IDPs) from protein sequences. 

In [319]:
import numpy as np
import json

import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Data loading

In [249]:
def load_sequences(file_path, idp_file_path, output_file_path):
    protein_data = []

    idp_proteins = []
    with open(idp_file_path, 'r') as file:
        idp_proteins = {prot_id.strip() for prot_id in file}

    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i >= 5570:
                break
            parts = line.split()
            protein_id = parts[0]
            sequence = ''.join(parts[2:])
            protein_class = "idp" if protein_id in idp_proteins else "non-idp"
            protein_data.append({
                'id': protein_id,
                'sequence': sequence.replace('*', ''),
                'class': protein_class
            })

    idp_n = idp_file_path.split('_')[-1].split('.')[0]
    idp_n_control = sum(1 for prot_id in protein_data if prot_id['class'] == 'idp')
    print(f'Expected: {idp_n}, Found: {idp_n_control}')

    with open(output_file_path, 'w') as file:
        json.dump(protein_data, file, indent=4)

    return protein_data

In [250]:
yeast_proteins = load_sequences('data/yeast_sequences.txt', 'data/disprot_yeast_194.txt', 'data/yeast_proteins.json')

Expected: 194, Found: 194


### Sequence based feature extraction

ProtIDR based feature extraction

In [251]:
def protseq_to_binseq(protein_sequence):
    protein_sequence = list(protein_sequence)
    DPaa = ['A', 'R', 'G', 'Q', 'S', 'P', 'E', 'K']
    OPaa = ['N', 'W', 'C', 'I', 'F', 'Y', 'V', 'L', 'M', 'D', 'H', 'T']

    for i, residue in enumerate(protein_sequence):
        if residue in DPaa:
            protein_sequence[i] = 1
        elif residue in OPaa:
            protein_sequence[i] = 0

    return protein_sequence


def get_protidr_vector(protein_sequence):
    # step 1 - protein sequence to binary sequence
    binary_sequence = protseq_to_binseq(protein_sequence)

    # step 2 - iad vector construction
    iad1_vector = []
    iad0_vector = []
    current_iad1 = 0
    found_one = False

    current_iad0 = 0
    found_zero = False

    for i in binary_sequence:
        if i == 1:
            if found_one: 
                iad1_vector.append(current_iad1)
            current_iad1 = 0
            found_one = True
        else:
            current_iad1 += 1
    
        if i == 0:
            if found_zero: 
                iad0_vector.append(current_iad0)
            current_iad0 = 0
            found_zero = True
        else:
            current_iad0 += 1
    
    # step 2.1 and 2.2: frequency histograms
    iad1_hist, _ = np.histogram(iad1_vector, bins=5, range=(0, 14))
    iad0_hist, _ = np.histogram(iad0_vector, bins=5, range=(0, 14))

    # Step 2.3: frequency histograms to probability distributions
    iad1_probs = iad1_hist / len(iad1_vector) * 100
    iad0_probs = iad0_hist / len(iad0_vector) * 100

    # ProtIDR vector
    protidr_vector = np.append(iad0_probs, iad1_probs)

    return protidr_vector

ProtPCV2 based feature extraction

ProtPCV2 vector is constructed following steps mentioned in the paper [ProtPCV](https://link.springer.com/article/10.1007/s12539-020-00380-w)

In [268]:
def protseq_to_binseq2(protein_sequence, pcp):
    protein_sequence = list(protein_sequence)
    HC_a = ['F','W','Y','H']
    HC_ap = ['A','M','C','L','V','I','D','E','N','Q','S','T','R','K','P','G']

    PO_p = ['D','E','H','K','N','Q','R','S','T','Y']
    PO_np = ['A','C','F','G','I','L','M','P','W','V']
    
    CC_nz = ['C','I','D','E','N','Q','Y','S','T','R','K','H','P','W','G'] 
    CC_z = ['A','F','L','M','V']
    
    HI_hb = ['A','M','C','F','L','V','I']
    HI_hp = ['D','E','N','Q','Y','S','T','R','K','H','P','W','G']
    
    HY_po = ['A','C','F','G','I','P','W','S','T','L','V']
    HY_ne = ['D','E','Q','Y','R','K','H','M','N']

    for i, residue in enumerate(protein_sequence):
        if pcp == "HC":
            if residue in HC_a:
                protein_sequence[i] = 1
            elif residue in HC_ap:
                protein_sequence[i] = 0
        elif pcp == "PO":
            if residue in PO_p:
                protein_sequence[i] = 1
            elif residue in PO_np:
                protein_sequence[i] = 0
        elif pcp == "CC":
            if residue in CC_nz:
                protein_sequence[i] = 1
            elif residue in CC_z:
                protein_sequence[i] = 0
        elif pcp == "HI":
            if residue in HI_hb:
                protein_sequence[i] = 1
            elif residue in HI_hp:
                protein_sequence[i] = 0
        elif pcp == "HY":
            if residue in HY_po:
                protein_sequence[i] = 1
            elif residue in HY_ne:
                protein_sequence[i] = 0

    return protein_sequence


def get_protpcv2_vector(protein_sequence):
    pcp_list = ["HC", "PO", "CC", "HI", "HY"]
    pcp_list = ["PO", "HC", "HI", "CC", "HY"]
    protpcv2 = []

    for pcp in pcp_list:
        # step 1 - protein sequence to binary sequence
        binary_sequence = protseq_to_binseq2(protein_sequence, pcp)

        # step 2 - iad vector construction
        iad1_vector = []
        iad0_vector = []
        current_iad1 = 0
        found_one = False

        current_iad0 = 0
        found_zero = False

        for i in binary_sequence:
            if i == 1:
                if found_one: 
                    iad1_vector.append(current_iad1)
                current_iad1 = 0
                found_one = True
            else:
                current_iad1 += 1
        
            if i == 0:
                if found_zero: 
                    iad0_vector.append(current_iad0)
                current_iad0 = 0
                found_zero = True
            else:
                current_iad0 += 1
        
        # step 2.1 and 2.2: frequency histograms
        iad1_hist, _ = np.histogram(iad1_vector, bins=5, range=(0, 14))
        iad0_hist, _ = np.histogram(iad0_vector, bins=5, range=(0, 14))
        

        # Step 2.3: frequency histograms to probability distributions
        iad1_probs = np.where(iad1_hist == 0, 0, iad1_hist / len(iad1_vector)) * 100
        iad0_probs = np.where(iad0_hist == 0, 0, iad0_hist / len(iad0_vector)) * 100
        # iad1_probs = iad1_hist / len(iad1_vector) * 100
        # iad0_probs = iad0_hist / len(iad0_vector) * 100

        # ProtIDR vector
        protpcv2.append(np.append(iad1_probs, iad0_probs))
        
    protpcv2 = np.concatenate(protpcv2)
    return protpcv2

CIDER based feature extraction

10 parameters:
- fraction of negative charge
- fraction of positive charge from Das-Pappu phase diagram
- fraction of charged residues
- net charge per residue
- kappa
- omega
- sigma
- delta
- max delta
- hydropathy

In [269]:
from localcider.sequenceParameters import SequenceParameters

def get_cider_vector(protein_sequence):
    seq = SequenceParameters(protein_sequence)

    cider_vector = np.array([
        seq.get_fraction_negative(),
        seq.get_fraction_positive(),
        seq.get_FCR(),
        seq.get_NCPR(),
        seq.get_kappa(),
        seq.get_Omega(),
        # sigma (σ) = (f+-f-)2/(f++f-) 
        (seq.get_fraction_positive() - seq.get_fraction_negative())**2 / (seq.get_fraction_positive() + seq.get_fraction_negative()),
        seq.get_delta(),
        seq.get_deltaMax(),
        seq.get_mean_hydropathy(),
    ])

    return cider_vector

### Design and implementation of IDPpred

In [320]:
class IDPClassifier:
    def __init__(self, input_dim, hidden_layers, output_dim=2, activation='relu', lr=0.001, loss=tf.keras.losses.SparseCategoricalCrossentropy()):
        self.model = tf.keras.Sequential()

        self.model.add(Dense(input_dim, activation=activation, input_shape=(input_dim,)))  # input layer
        
        for num_neurons in hidden_layers:
            self.model.add(Dense(num_neurons, activation=activation))  # hidden layers
        self.model.add(Dense(output_dim, activation='sigmoid'))  # output layer

        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        self.model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=100, batch_size=32, class_weight=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, class_weight=class_weight)

    def predict(self, X):
        return self.model.predict(X)

    def save(self, path):
        self.model.save_weights(path)

    def load(self, path):
        self.model.load_weights(path)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)

Data preprocessing

In [307]:
X = []
y = []
for protein in yeast_proteins:
    X.append(protein['sequence'])
    y.append(1 if protein['class'] == 'idp' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [308]:
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))

In [309]:
print(f'train: {len(X_train)}')
print(f'test: {len(X_test)}')

print(f'idp in train: {sum(y_train)}')
print(f'idp in test: {sum(y_test)}')

train: 4456
test: 1114
idp in train: 155
idp in test: 39


Training

In [310]:
classifiers = ["protpcv2", "protidr"]
#classifiers = ["cider"]

In [ ]:
protpcv2_classifier = IDPClassifier(input_dim=50, hidden_layers=[40, 20, 10, 5], lr=1e-3)
protidr_classifier = IDPClassifier(input_dim=10, hidden_layers=[20, 10], lr=1e-3)
cider_classifier = IDPClassifier(input_dim=10, hidden_layers=[20, 10], lr=1e-3)

for classifier in classifiers:
    if classifier == "protpcv2":
        X_train_transformed = np.vstack([get_protpcv2_vector(protein) for protein in X_train])
        X_test_transformed = np.vstack([get_protpcv2_vector(protein) for protein in X_test])
    elif classifier == "protidr":
        X_train_transformed =  np.vstack([get_protidr_vector(protein) for protein in X_train])
        X_test_transformed =  np.vstack([get_protidr_vector(protein) for protein in X_test])
    elif classifier == "cider":
        X_train_transformed = np.vstack([get_cider_vector(protein) for protein in X_train])
        X_test_transformed = np.vstack([get_cider_vector(protein) for protein in X_test])
    
    if classifier == "protpcv2":
        epochs = 600
        print("Training protpcv2 classifier")
        protpcv2_classifier.train(X_train_transformed, np.array(y_train), epochs=epochs, batch_size=128, class_weight=class_weights)
        protpcv2_classifier.save(f"./models/{classifier}_classifier_{epochs}_v2.weights.h5")
        print("test loss, test acc:", protpcv2_classifier.evaluate(X_test_transformed, np.array(y_test)))
    elif classifier == "protidr":
        epochs = 300
        print("Training protidr classifier")
        protidr_classifier.train(X_train_transformed, np.array(y_train), epochs=epochs, batch_size=128, class_weight=class_weights)
        protidr_classifier.save(f"./models/{classifier}_classifier_{epochs}_v2.weights.h5")
        print("test loss, test acc:", protidr_classifier.evaluate(X_test_transformed, np.array(y_test)))
    elif classifier == "cider":
        epochs = 50
        print("Training cider classifier")
        cider_classifier.train(X_train_transformed,  np.array(y_train), epochs=epochs, batch_size=128, class_weight=class_weights)
        cider_classifier.save(f"./models/{classifier}_classifier_{epochs}.weights.h5")
        print("test loss, test acc:", cider_classifier.evaluate(X_test_transformed, np.array(y_test)))

In [ ]:
protpcv2_classifier = IDPClassifier(input_dim=50, hidden_layers=[40, 20, 10, 5])
protidr_classifier = IDPClassifier(input_dim=10, hidden_layers=[20, 10])
cider_classifier = IDPClassifier(input_dim=10, hidden_layers=[20, 10])

protpcv2_classifier.load("./models/protpcv2_classifier_600_v2.weights.h5")
protidr_classifier.load("./models/protidr_classifier_300_v2.weights.h5")
cider_classifier.load("./models/cider_classifier_50.weights.h5")

def idp_pred(protein_sequence):
    # feature extraction
    protpcv2 = get_protpcv2_vector(protein_sequence)
    protidr = get_protidr_vector(protein_sequence)
    cider = get_cider_vector(protein_sequence)

    # initial prediction
    protpcv2_pred = protpcv2_classifier.predict(protpcv2.reshape(1, -1))
    protidr_pred = protidr_classifier.predict(protidr.reshape(1, -1))
    cider_pred = cider_classifier.predict(cider.reshape(1, -1))
    # print(protpcv2_pred, protidr_pred, cider_pred)

    protpcv2_pred = 1 if protpcv2_pred[0][1] > protpcv2_pred[0][0] else 0
    protidr_pred = 1 if protidr_pred[0][1] > protidr_pred[0][0] else 0
    cider_pred = 1 if cider_pred[0][1] > cider_pred[0][0] else 0
    # print(protpcv2_pred, protidr_pred, cider_pred)

    # voting
    final_prod = 1 if protpcv2_pred + protidr_pred + cider_pred >= 2 else 0

    return final_prod

In [327]:
test_idp = [protein for i, protein in enumerate(X_test) if y_test[i] == 1]

In [ ]:
true_positives = 0

for protein in test_idp:
    if idp_pred(protein) == 1:
        true_positives += 1

print(f'TP = {true_positives}; FP = {len(test_idp) - true_positives}')

In [329]:
# 600, 300, 50 --> TP = 18, FP = 21
# 600, 300, 50 (v2) --> TP = 24, FP = 15

In [330]:
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

# true positive rate, sensitivity
def tpr(tp, fn):
    return tp / (tp + fn) # true positive / actual positive

# true negative rate, specificity
def tnr(tn, fp):
    return tn / (tn + fp) # true negative / actual negative

# balanced accuracy
def bac(tpr, tnr):
    return (tpr + tnr) / 2

In [ ]:
for x_test, y_test in zip(X_test, y_test):
    pred = idp_pred(x_test)
    
    if y_test == 1 and pred == 1:
        true_positives += 1
    elif y_test == 0 and pred == 0:
        true_negatives += 1
    elif y_test == 0 and pred == 1:
        false_positives += 1
    elif y_test == 1 and pred == 0:
        false_negatives += 1

In [332]:
print("TP:", true_positives)
print("TN:", true_negatives)
print("FP:", false_positives)
print("FN:", false_negatives)

tpr_val = tpr(true_positives, false_negatives)
tnr_val = tnr(true_negatives, false_positives)

print("TPR:", tpr_val)
print("TNR:", tnr_val)
print("BAC:", bac(tpr_val, tnr_val))

TP: 24
TN: 589
FP: 486
FN: 15
TPR: 0.6153846153846154
TNR: 0.5479069767441861
BAC: 0.5816457960644008
